In [1]:
try:
    import psycopg2
    import psycopg2.extras
    import pandas as pd
    import opendp
    from opendp.transformations import *
    from opendp.measurements import *
    from opendp.mod import enable_features
    enable_features("contrib")
    from opendp.mod import binary_search
    from opendp.accuracy import discrete_laplacian_scale_to_accuracy
    
    # Import time module and logging
    import time
    import logging
    #logging.basicConfig(filename='code_execution.log', level=logging.INFO, format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    import os
    log_file_path = os.path.join(os.getcwd(), 'code_execution.log')
    logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    print("Current Working Directory:", os.getcwd())

    print("libraries imported")
except ImportError as e:
    print(f"Error importing libraries: {e}")

Current Working Directory: /Users/anhpham/Projects/verse_project
libraries imported


In [2]:
def postgres_DB(dbname, user, host, password):
    try:
        # Connect to the database
        conn = psycopg2.connect(dbname=dbname, user=user, host=host, password=password)
        try:
            # Create a cursor
            cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
            print("Connected to database, cursor defined")
            return conn, cur
        except psycopg2.Error as e:
            print(f"Error creating cursor: {e}")
            # Close the connection in case of an error
            conn.close()
            return None, None      
    except psycopg2.Error as e:
        print(f"Error connecting to database: {e}")
        return None, None

In [3]:
def csv_scan(first_file):
    #Scanning CSV's into dataframes
    df = pd.read_csv(first_file)
    return df

In [4]:
def postgres_scan(table_name, cursor):
    # Execute a query to fetch column names and data from the table
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)

    # Fetch column names from the cursor description
    column_names = [desc[0] for desc in cursor.description]

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(cursor.fetchall(), columns=column_names)

    return df

In [5]:
def filter_df(df, condition):
    # Filter the DataFrame based on the given condition
    filtered_df = df.query(condition)
    # Reset index in the new dataframe
    filtered_df.reset_index(drop=True, inplace=True)

    return filtered_df

In [6]:
def pad_data(df, filtered_df, track_column):
    # Find the maximum number of rows in the original DataFrame
    max_rows = len(df)

    # Find the difference in the number of rows
    num_rows_diff = len(df) - len(filtered_df)

    # Create a DataFrame of zeros with the same columns as low_gpa_df
    zeros_df = pd.DataFrame(0, index=range(num_rows_diff), columns=filtered_df.columns)

    # Concatenate low_gpa_df and zeros_df vertically
    df_padded = pd.concat([filtered_df, zeros_df],ignore_index=True)

    # Create a dummy column to track zero-filled rows
    # .astype(int)converts boolean TRUE/FALSE to 1-0
    # 0 for False(fake data), 1 for True(real data)
    zero_filled = ~(df_padded == 0).all(axis=1)
    df_padded[track_column] = zero_filled.astype(int)

    return df_padded

In [7]:
#Partially pad so that the data contains k*100 % dummy values
def DP_pad_data(filtered_df, original_df, k, track_column):
#     if not isinstance(filtered_df, pd.DataFrame) or not isinstance(original_df, pd.DataFrame):
#         raise ValueError("Both filtered_df and original_df should be valid DataFrames.")
    min_size = len(filtered_df)
    max_size = len(original_df)
    if min_size >= max_size:
        raise ValueError("filtered_df should have fewer rows than original_df.")
    size_diff = max_size - min_size
    
    if not 0 < k < 1:
        raise ValueError("The value of k should be between 0 and 1.")
        
    # Calculate the number of rows to pad, but ensure it does not exceed size_diff 
    # to satisfy k% dummy rows in the total size, this is the cross-formula, this could grow past max_size sometimes
    n_rows=int((min_size * k)/(1-k))
    
    #when total number of rows after pad reaches max_size, meaning n_rows=size_diff #cross maths
    max_k = size_diff / (min_size + size_diff)
    
    if n_rows > size_diff:
        print(f"Sorry, padding up to {k*100:.2f}% dummy in this dataset is not applicable, as it will exceed the size of input data.\n"
              f"Your data will be padded to the maximum input size of {max_size}.")
        print(f"Maximum effective k: {max_k:.3f}, beyond this value the data will be padded to the maximum size.\n"
              f"Runtime is now equal to fully private count.")
        
    #realistic pad rows might be different because we cannot exceed input (original datasize)
    # the min funct ensures padding only til the max allowed
    pad_rows = min(n_rows, size_diff)
    size_df_padded = min_size + pad_rows
    
    zeros_df = pd.DataFrame(0, index=range(pad_rows), columns=filtered_df.columns)
    df_padded = pd.concat([filtered_df, zeros_df], ignore_index=True)

    print(f"Size padded such that the dataset has {k*100:.2f}% dummy data: {df_padded.shape[0]}")
    print()

    
    zero_filled = ~(df_padded == 0).all(axis=1)
    df_padded[track_column] = zero_filled.astype(int)
    
    return df_padded

In [8]:
def join(df, df2, merge_type, column_compare):
    final_df = pd.merge(left = df, right = df2, how = merge_type, on = column_compare)
    return final_df

In [9]:
#count the number of real data (not padded) in a padded dataframe -> count that hides execution time
def count_real(padded_df, track_column):
    count_real = (padded_df[track_column] == 1).sum()    
    #convert to int (bc how pandas handle boolean)
    return count_real.item()  

In [10]:
def mean_real(df, column): 
    #later worry about which df
    # Check if the column exists in the DataFrame
    if column in df.columns:
        # Calculate the mean for the specified column
        mean_value = df[column].mean()
        return mean_value
    else:
        raise ValueError("Column '{}' does not exist in the DataFrame.".format(column_name))

In differential privacy, sensitivity, epsilon, and scale are important concepts used in privacy-preserving mechanisms to control the amount of noise added to query results and ensure privacy protection.

Sensitivity:
Sensitivity refers to the maximum possible change in the query result when a single individual's data is added or removed from the dataset. It measures how much impact a single individual can have on the query result. The sensitivity value is specific to each query and dataset.

Epsilon (ε):
Epsilon is a privacy parameter that controls the level of privacy protection provided by a differentially private mechanism. A smaller value of epsilon provides stronger privacy guarantees but may result in noisier query results. On the other hand, a larger value of epsilon provides weaker privacy guarantees but yields less noisy query results.

The range of epsilon (ε) in differential privacy is typically between 0 and infinity. However, in practice, epsilon is always a positive value, and it's common to use epsilon values that are greater than or equal to 0 and less than or equal to 1.

A smaller epsilon value provides stronger privacy guarantees, meaning that the released data is more protected and less likely to reveal sensitive information about individuals in the dataset. On the other hand, a larger epsilon value provides weaker privacy guarantees, meaning that the released data may be more accurate but less private.

The choice of epsilon depends on the specific privacy requirements and the level of privacy protection needed for the application. In general, smaller epsilon values are preferred for scenarios where strong privacy protection is essential, such as medical or financial data. However, larger epsilon values can be used for scenarios where a balance between privacy and data accuracy is acceptable.

It's important to note that as epsilon increases, the amount of noise added to the query result decreases, which means the released data becomes less private. Therefore, the selection of an appropriate epsilon value requires a careful trade-off between privacy and utility (accuracy of the released data).

Scale:
Scale is a parameter used in the Laplace and Gaussian mechanisms to determine the amount of noise added to the query result. For the Laplace mechanism, the scale is directly proportional to sensitivity/epsilon, while for the Gaussian mechanism, the scale is sensitivity/epsilon. A higher scale value means more noise is added, which results in more privacy protection.

In [11]:
def mechanism_selection():
    print("Select a mechanism for adding noise:")
    print("1. Laplace Noise")
    print("2. Gaussian Noise")
    print("3. Random Noise")
    
    while True:
        choice = input("Enter your choice (1, 2, or 3): ")
        if choice == "1":
            return "laplace"
        elif choice == "2":
            return "gaussian"
        elif choice == "3":
            return "random"
        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

In [12]:
def privacy_parameters():
    print("In order to add noise using Laplace, Gaussian or Random mechanism, first determine privacy parameters.")
    print("Enter the sensitivity (e.g., 1): ")
    sensitivity = float(input())
    print("Enter the epsilon (e.g., 0.1): ")
    epsilon = float(input())
    # Calculate the scale parameter for Laplace distribution
    scale = sensitivity / epsilon
    return sensitivity, epsilon, scale

In [13]:
def add_noise(result, mechanism, scale):
    if mechanism == "laplace":
        return laplace_noise(result,scale)
    elif mechanism == "gaussian":
        return gaussian_noise(result,scale)
    elif mechanism == "random":
        return random_noise(result,scale)
    else:
        raise ValueError("Invalid mechanism choice. Supported mechanisms are 'laplace' and 'gaussian'.")

The result is a random draw from the discrete Laplace/Gaussian distribution, centered at the true count of the number of records in the underlying dataset.

In [14]:
# Add Laplace noise for differential privacy
def laplace_noise(result, scale):
    #scale here just has to be hard-coded. Because scale is proportional but not equal to sensitivity/epsilon
    #there isn't a universal way I've found to calculate scale for laplace without knowing standard deviation.
    if isinstance(result, int):
        dp_count = make_base_discrete_laplace(scale) 
    if isinstance(result, float):
        dp_count = make_base_laplace(scale) 
    noisy_result = dp_count(result)
    return noisy_result

In [15]:
def gaussian_noise(result,scale):
    if isinstance(result, int):
        dp_count = make_base_discrete_gaussian(scale)
    if isinstance(result, float):
        dp_count = make_base_gaussian(scale) 
    noisy_result = dp_count(result)
    return noisy_result

In [16]:
import numpy as np

def random_noise(result, scale):
    # Generate Laplace noise and add it to the result
    #scale is hard-coded, explanation above
    if isinstance(result, int):
        noisy_result = int(result + np.random.laplace(loc=0, scale=scale))
    if isinstance(result, float):
        noisy_result = float(result + np.random.laplace(loc=0, scale=scale))
    return noisy_result

In [17]:
def max_deviation(scale, alpha=None):
    if alpha is None:
        alpha = float(input("Enter the alpha (e.g., 0.05): "))

    max_deviation = discrete_laplacian_scale_to_accuracy(scale=scale, alpha=alpha)
    return max_deviation

In [18]:
def read_log_file(log_file_path):
    print("Trying to read log file:", log_file_path)
    try:
        if os.path.exists(log_file_path):
            with open(log_file_path, 'r') as log_file:
                log_contents = log_file.read()
            return log_contents
        else:
            return f"Log file '{log_file_path}' not found."
    except Exception as e:
        return f"Error occurred while reading the log file: {e}"

In [19]:
import gspread
from gspread_dataframe import set_with_dataframe

def parse_log_content(log_content):
    parsed_data = []

    for line in log_content.split("\n"):
        if line.strip():
            try:
                # Split based on "-"
                timestamp_str, event_duration_str = line.split(" - ", 1)
                # Remove trailing white space
                timestamp = timestamp_str.strip()
                # Split the event and duration string with a maximum of 1 split
                event, *duration_str_parts = event_duration_str.split(": ", 1)
                # Join the duration string parts back together in case there were additional separators in the event description
                duration_str = ": ".join(duration_str_parts).strip()

                if not duration_str:
                    raise ValueError(f"Invalid log format in line: {line}")

                # Check if the duration includes units (e.g., 'ms' or 'seconds')
                if 'ms' in duration_str:
                    duration = float(duration_str.split(" ms")[0])
                elif 'seconds' in duration_str:
                    duration = float(duration_str.split(" seconds")[0]) * 1000  # Convert seconds to milliseconds
                else:
                    raise ValueError(f"Invalid duration format in line: {line}")

                # Extract the value of 'k' from the event description
                k_str = event.split("k=")[1]
                k = float(k_str[:-1])  # Remove '%' and convert to a float

                parsed_data.append({"Timestamp": timestamp, "Event": event, "k": k, "Duration": duration})
            except ValueError as e:
                # Handle the case when the line does not match the expected format, here its the separator -------
                print(f"Warning: Skipping line due to invalid format: {line}")

    return parsed_data

In [20]:
import warnings
import re

def write_log_GS(parsed_data):
    try:
        # Create a DataFrame from the parsed data
        log_df = pd.DataFrame(parsed_data)

        # Extract the 'k' value from the 'Event' column and create a new 'k' column
        log_df['k'] = log_df['Event'].str.extract(r'<k=(\d+\.\d+)>')

        # Remove the <k=0.25> part from the 'Event' column
        log_df['Event'] = log_df['Event'].str.replace(r'<k=\d+\.\d+>', '', regex=True)

        # Reorder the columns to match the desired order: "Timestamp", "Event", "k", "Duration"
        log_df = log_df[["Timestamp", "Event", "k", "Duration"]]

        # Rename the columns to match the desired column names
        log_df = log_df.rename(columns={"Timestamp": "Time", "Event": "Event", "k": "k", "Duration": "Duration (in ms)"})

        # Authenticate with Google Sheets using the credentials JSON file
        gc = gspread.service_account(filename='verse-project-timesheet-f4792514dd22.json')

        # Open the Google Sheet by its title
        sheet = gc.open('Code Execution Time Log')

        # Select the worksheet to put the data
        worksheet = sheet.worksheet('Sheet1')

        # Create a list of column names
        column_names = log_df.columns.tolist()

        # Clear the existing data in the worksheet and write the column names
        worksheet.clear()
        worksheet.append_row(column_names)

        # Convert the DataFrame to a list of lists (values) for writing to the Google Sheet
        values = log_df.values.tolist()

        # Write the values to the Google Sheet, starting from cell A2 (after the column names)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            worksheet.update('A2', values, value_input_option='USER_ENTERED')

        print("Log data has been written to the Google Sheet.")
    except Exception as e:
        print(f"An error occurred while writing data to Google Sheet: {e}")
        print("Error details:")
        import traceback
        traceback.print_exc()
    

In [21]:
def log_execution_time(event_data):
    for event, k, time_ms in event_data:
        logging.info(f'{event} <k={k}>: {time_ms:.3f} ms')

In [22]:
def main():
    # Connect to the Postgress database and create cursor specific to that database
    conn, cur = postgres_DB('university', 'anhpham', 'localhost', 'freedom')
    # Check if the connection and cursor are valid
    if conn and cur:  
        try:
            '''
            GPA < 3 QUERY
            '''
            #"""
            print()
            print("1. SELECT COUNT(*) FROM student WHERE gpa < 3;")

            # record start time
            start_scan = time.time()
            
            # SCAN: Specify the table name to fetch data -> Original Dataframe
            table_name = 'student'
            df = postgres_scan(table_name, cur)
            
            # record end time
            end_scan = time.time()
            scan_time = (end_scan-start_scan) * 10**3
            
            # record start time
            start_filter = time.time()
        
            #FILTER daframe
            filtered_column = 'gpa'
            condition = f"{filtered_column} <3"

            low_gpa_df = filter_df(df, condition)
            
            # record end time
            end_filter = time.time()
            filter_time = (end_filter-start_filter) * 10**3
            
            # record start time
            start_public=time.time()
            
            #FULLY PUBLIC 
            gpa_count = low_gpa_df.shape[0]
            
            # record end time
            end_public = time.time()
            public_time = (end_public-start_public) * 10**3
            
            # record start time
            start_pad = time.time()
            
            #PAD data
            low_gpa_padded=pad_data(df,low_gpa_df, "dummy")
            #print(low_gpa_padded)
            
            # record end time
            end_pad = time.time()
            pad_time = (end_pad-start_pad) * 10**3
            
            # record start time
            start_priv=time.time()

            #FULLY PRIVATE
            # Call count_real function -> hide execution time when count            
            real_count = count_real(low_gpa_padded, 'dummy') 
            print("Real result is:", real_count)
            print()
            
            # record end time
            end_priv = time.time()
            full_priv_time = (end_priv-start_priv) * 10**3
            
            #DIFFERENTIAL PRIVATE PAD
            k=0.25
            dp_padded = DP_pad_data(low_gpa_df,df, k, "dummy")
            print(dp_padded)
            print()
            
            # record start time
            start_dp = time.time()
            
            #get real result from this DP padded data, it should still be the same as real result
            dp_padded_count = count_real(dp_padded, 'dummy')
            
            # record end time
            end_dp = time.time()
            dp_padded_time = (end_dp-start_dp) * 10**3
            
            #Add noise to DP Pad
            
             #Get privacy parameters for adding noise
            sensitivity, epsilon, scale = privacy_parameters()
            print()
            
            count_mechanism = mechanism_selection()
            print(f"Selected mechanism: {count_mechanism.capitalize()} Noise")           

            #Add DP noise based on the selected mechanism
            dp_result = add_noise(real_count, count_mechanism, scale)
            print("DP Count with", count_mechanism, "is:", dp_result)
            
            count_deviation=max_deviation(scale, 0.05)
            print("Max deviation for count at a 95% confidence level: {:.3f}".format(count_deviation))
            print(f"When the {count_mechanism} distribution’s scale is {scale},\nthe DP estimate differs from the exact estimate by no more than {count_deviation:.3f} at a 95% confidence level.")
            print() 

            # print the difference between start and end time in milli. secs
            print("TIME to 3 decimal ")        
            events = [
                ('Scan', k, scan_time),
                ('Filter', k, filter_time),
                ('Pad', k, pad_time),
                ('Fully Public', k, public_time),
                ('Fully Private', k, full_priv_time),
                ('DP Private', k, dp_padded_time)
            ]

            # logging the time duration of each event
            log_execution_time(events)
            logging.info('-' * 38)
            
            # Open and read log
            #log_file_path = 'code_execution.log'
            log_content = read_log_file(log_file_path)
            print(f"log content\n{log_content}")
            # Parse the log content to a structured format
            parsed_data = parse_log_content(log_content)

            # Call the write_log_GS function with the parsed data
            write_log_GS(parsed_data)

            # Print the log data for verification
            print("Log Data:")
            for entry in parsed_data:
                print("PARSED DATA", entry)

            #"""
            
            '''
            MEAN GPA FROM STUDENT
            '''
            
            """
            print()
            print("2. SELECT AVG(gpa) AS average_gpa FROM student;")
            
            mean_gpa = mean_real(df,"gpa")
            print("Real result is:", mean_gpa)
            print()
            
            sensitivity, epsilon, scale = privacy_parameters()
            print()
            
            mean_mechanism = mechanism_selection()
            print(f"Selected mechanism: {mean_mechanism.capitalize()} Noise") 
            
            dp_mean=add_noise(mean_gpa, mean_mechanism, scale)
            print("DP Mean GPA with {} is: {:.3f}".format(mean_mechanism, dp_mean))
            
            mean_deviation=max_deviation(scale, 0.05)
            #print("Max deviation for mean at a 95% confidence level: {:.3f}".format(mean_deviation))
            print(f"Max deviation for mean at a 95% confidence level: {mean_deviation:.3f}")
            print("When the {} distribution’s scale is {},\nthe DP estimate differs from the exact estimate by no more than {:.3f} at a 95% confidence level.".format(mean_mechanism, scale, mean_deviation))
            print()
            
            print("TIME")
            #NEED TO limit THE MEAN VALUE BETWEEN 0-4, working on that right now
             
            """
            
            '''
            NUMPHDS > 65
            '''
            
            """
            print()
            print("3. SELECT count(*) FROM course, dept WHERE course.dname = dept.dname AND numphds > 65")
            
            table_name = 'course'
            df1 = postgres_scan(table_name, cur)
            
            table_name = 'dept'
            df2 = postgres_scan(table_name, cur)
            
            #Filter dataframe
            filtered_column = 'numphds'
            condition = f"{filtered_column} >65"

            df_filtered = filter_df(df2, condition)
            
            df_joined = join(df1, df_filtered, 'inner', 'dname')
            
            #Pad data
            large_PHD_padded=pad_data(df2, df_joined, "dummy")
            
            # Call count_real function -> hide execution time when count
            real_result = count_real(large_PHD_padded, 'dummy') 
            print("Real result is:", real_result)
            print()
            
            #Get privacy parameters
            sensitivity, epsilon, scale = privacy_parameters()

            join_mechanism = mechanism_selection()
            print(f"Selected mechanism: {join_mechanism.capitalize()} Noise")
            print()

            # Add DP noise based on the selected mechanism
            dp_result = add_noise(real_result, join_mechanism, scale)
            print("DP result with", join_mechanism, "is:", dp_result)
            
            join_deviation = max_deviation(scale, 0.05)
            print("Max deviation for count join at a 95% confidence level: {:.3f}".format(join_deviation))
            print("When the {} distribution’s scale is {},\nthe DP estimate differs from the exact estimate by no more than {:.3f} at a 95% confidence level.".format(join_mechanism, scale, join_deviation))
            """
             
        finally:
            cur.close()
            conn.close()
            print()
            print("Connection to database closed. Users can repeat the process or try with another database after this.")
            
    #start writing csv code here without interfering with postgress code   
    
    #scan data
    '''
    # record start time
    start_scan = time.time()
    
    df = csv_scan("1KB.csv")
    print(df)
    
    # record end time
    end_scan = time.time()
    scan_time = (end_scan-start_scan) * 10**3
    
    # record start time
    start_filter = time.time()
        
    #filter
    filtered_column = 'Age'
    condition = f"{filtered_column} < 26"

    young_df = filter_df(df, condition)
    print(young_df)
    
    # record end time
    end_filter = time.time()
    filter_time = (end_filter-start_filter) * 10**3
    
    # record start time
    start_public=time.time()
    
    #count
    #FULLY PUBLIC 
    young_count = young_df.shape[0]

    # record end time
    end_public = time.time()
    public_time = (end_public-start_public) * 10**3

    # record start time
    start_pad = time.time()

    #PAD data
    young_padded=pad_data(df,young_df, "dummy")
    #print(low_gpa_padded)

    # record end time
    end_pad = time.time()
    pad_time = (end_pad-start_pad) * 10**3
    
    # record start time
    start_priv=time.time()

    #FULLY PRIVATE
    # Call count_real function -> hide execution time when count            
    real_count = count_real(young_padded, 'dummy') 
    print("Real result is:", real_count)
    print()

    # record end time
    end_priv = time.time()
    full_priv_time = (end_priv-start_priv) * 10**3
    
    '''
            
    
       
if __name__ == "__main__":
    main()

Connected to database, cursor defined

1. SELECT COUNT(*) FROM student WHERE gpa < 3;
Real result is: 40

Size padded such that the dataset has 25.00% dummy data: 53

    sid                 sname sex  age  year  gpa  dummy
0     4     Sulfate, Barry M.   m   19     2  2.8      1
1     7    Sather, Roberto B.   m   22     4  2.2      1
2     9       Smith, Joyce A.   f   21     4  2.0      1
3    13         Kellerman, S.   f   21     3  2.9      1
4    17     Thorton, James Q.   m   28     4  2.7      1
5    18                 Gooch   m   26     1  1.4      1
6    19             Smith, L.   m   43     4  0.7      1
7    21              Surk, K.   m   23     2  2.5      1
8    22             Emile, R.   m   18     1  2.0      1
9    29          Hamilton, S.   m   21     3  2.8      1
10   31           Andrews, R.   m   19     2  2.8      1
11   34     Kasten, Norman L.   m   23     2  2.5      1
12   38              Auen, B.   m   21     3  2.7      1
13   40         Rosemeyer, S.   f  

In [23]:

#write some small program to increase file size
#to get data points for the x-bar


#scan, filter, join, count
#extract 
#run individually
#just scan, just joint
#if 25% 50% percent dummy then how the performance change
#setting 25% dummy


#mechanism - getscale
#check DP pad
#about logging, check if my log is checking each stage? is that what we re trying to do
#try putting log in google sheet
#visualization
